In [1]:
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import defaultdict
import ast
from tqdm.notebook import tqdm as tqdm
import pickle
import contractions

c:\Users\srija\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Creating Inverted Index

In [31]:
def helper(inv_idx_freq, inv_idx_postings, set_of_tokens, docID):
    for j in set_of_tokens:
        inv_idx_postings[j].append(docID)
        inv_idx_freq[j] = inv_idx_freq[j] + 1
        
        

In [32]:
inv_idx_freq  = defaultdict(int) # initializing the doc Frequency dictionary as a default dict which return 0 whenver a key does not exists.
inv_idx_postings  = defaultdict(list) # intializing the postings dictionary to return an empty list whenever it is invoked for a key that does not exits.
sample = 5
docIDs = []
folder_path = 'Q1_2_output'
for filename in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contents = file.read()
            docID = int(filename[-4:])
            docIDs.append(docID)
            # print(docID)
            # set_of_tokens = set(contents.strip('][').split(','))
            set_of_tokens = set(ast.literal_eval(contents))
            helper(inv_idx_freq, inv_idx_postings, set_of_tokens, docID)
            # print(f'{filename}: {contents}')
            # print(set_of_tokens)
            # sample -= 1
            # if (sample == 0):
            #     break


#sort the index in alphabetical order wrt terms
inv_idx_postings = dict(sorted(inv_idx_postings.items()))
inv_idx_freq = dict(sorted(inv_idx_freq.items()))

for key in inv_idx_postings:
    inv_idx_postings[key].sort() #sort the indivdual posting lists correponging to each term in the index


  0%|          | 0/1400 [00:00<?, ?it/s]

# Saving Inverted Index

In [33]:
pickle.dump(inv_idx_postings, open('inv_idx_postings.pkl', 'wb'))
pickle.dump(inv_idx_freq, open('inv_idx_freq.pkl', 'wb'))

# Inverted Index Operation Functions

In [2]:
def t1_and_t2(posting1, posting2):
  
  # if any of the postings is empty then their intersection will also be an empty list
  if len(posting1) == 0 or len(posting2) == 0:
    return [], 0

  ptr1 = 0 #pointer that iterates over posting 1
  ptr2 = 0 #pointer that iterates over posting 2

  comparisons = 0 #intialize number of comparison as 0
  ans = []


  while ptr1 < len(posting1) and ptr2 < len(posting2):
    comparisons += 1 
    if posting1[ptr1] ==  posting2[ptr2]: # this means that both the docs contains the word.
      ans.append(posting1[ptr1])
      ptr1 = ptr1 + 1
      ptr2 = ptr2 + 1

    elif posting1[ptr1] <  posting2[ptr2]:
      ptr1 = ptr1 + 1
    
    else:
      ptr2 = ptr2 + 1

  return ans, comparisons


In [3]:
# def not_t1(posting1):
#     global docIDs
#     ans = [] # to store all the docIDs except that are their in postings1

#     for i in docIDs:
#         if i in posting1:
#             continue
#         else:
#             ans.append(i)
#     return ans
    

In [4]:
# def t1_and_not_t2(posting1, posting2):
#     # if the first posting is empty then the intersection will also be an empty list
#     if len(posting1) == 0: 
#         return [], 0

#     if len(posting2) == 0:
#         return posting1, len(posting1)
    
#     # perform not query for posting2
#     not_posting2 = not_t1(posting2)

#     # now perform the and operation for posting1 and not_posting2
#     ans, comparisons = t1_and_t2(posting1, not_posting2)
#     return ans, comparisons
    
def t1_and_not_t2(posting1, posting2):
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons += 1

        if posting1[ptr1] == posting2[ptr2]: 
            ptr1+=1
            ptr2+=1
            continue
        elif posting1[ptr1] < posting2[ptr2]:
            ans.append( posting1[ptr1] )
            ptr1+=1

        else:
            ptr2+=1

    while ptr1 < len(posting1):
        ans.append(posting1[ptr1])
        ptr1+=1

    return ans, comparisons


In [5]:
def not_t1(posting1):
    global docIDs
    ans, comparisons = t1_and_not_t2(docIDs, posting1)
    return ans, comparisons

    

In [6]:
def t1_or_t2(posting1, posting2):

    #if any of the posting list is empty we will return the other list.
    if len(posting1) == 0 :
        return posting2, 0
    if len(posting2) == 0:
        return posting1, 0
        
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons+=1
        if posting1[ptr1] < posting2[ptr2]:
            ans.append(posting1[ptr1]) 
            ptr1+=1
        elif posting2[ptr2] < posting1[ptr1]  :
            ans.append(posting2[ptr2])
            ptr2+=1
        else:
            ans.append(posting2[ptr2])
            ptr1+=1
            ptr2+=1

    while(ptr1 < len(posting1)):
        ans.append( posting1[ptr1] )
        ptr1+=1
    
    while(ptr2 < len(posting2)):
        ans.append( posting2[ptr2] )
        ptr2+=1
    
    return ans, comparisons

In [7]:
def t1_or_not_t2(posting1, posting2):
    # here we need to take the not of posting2 and then do an or with posting 1

    if len(posting1) == 0: #if the first posting list is empty we will return the not of the other list.
        return not_t1(posting2)
    
    not_posting2, comparisons1 = not_t1(posting2)

    ans, comparisons = t1_or_t2(posting1, not_posting2)
    return ans, comparisons +  comparisons1

# Loading Inverted Index

In [10]:
inv_idx_postings = pickle.load(open("inv_idx_postings.pkl", 'rb'))
inv_idx_freq = pickle.load(open("inv_idx_freq.pkl", 'rb'))

# Query preprocessing

In [11]:
def remove_punc(token):
    translator = str.maketrans('', '', string.punctuation)
    return token.translate(translator)

In [12]:
def preprocess_query(query):
    content1 = query.lower()
    content1 = contractions.fix(content1)
    tokens = word_tokenize(content1)
    stop_words = set(stopwords.words("english"))
    tokens1 = [token for token in tokens if token not in stop_words]
    tokens2 = list(map(remove_punc, tokens1))
    tokens3 = [token for token in tokens2 if token.strip()]
    return tokens3

# Query Matching

In [22]:
docIDs=[ i for i in range(1,1401)]
doc_names=os.listdir("Q1_2_output")
queries=int(input())
for i in range(1,queries+1):
    query=input()
    query=preprocess_query(query)
    # print(query)
    operations=input().split(',')
    operations=[operation.strip() for operation in operations]
    # print(operations)
    q_ptr=1
    o_ptr=0
    answer=None
    if(query[0] in inv_idx_postings.keys()):
        answer=inv_idx_postings[query[0]]
    else:
        answer=[]
    comparisions=0
    prt_query=query[0]
    while(q_ptr<len(query) and o_ptr<len(operations)):
        prt_query+=" "+operations[o_ptr]+" "+query[q_ptr]
        T2=query[q_ptr]
        operation=operations[o_ptr]

        T2_postings=None
        if(T2 in inv_idx_postings.keys()):
            T2_postings=inv_idx_postings[T2]
        else:
            T2_postings=[]

        if operation=="AND":
            answer,temp=t1_and_t2(answer, T2_postings)
            comparisions+=temp
        elif operation=="OR":
            answer,temp=t1_or_t2(answer, T2_postings)
            comparisions+=temp
        elif operation=="AND NOT":
            answer,temp=t1_and_not_t2(answer, T2_postings)
            comparisions+=temp
        elif operation=="OR NOT":
            answer,temp=t1_or_not_t2(answer, T2_postings)
            comparisions+=temp
        q_ptr+=1
        o_ptr+=1
        # print(answer)
    print(f"Query {i}:",prt_query)
    print(f"Number of documents retrieved for query {i}:",len(answer))
    names=", ".join([doc_names[id-1]+".txt" for id in answer])
    print(f"Names of documents retrieved for query {i}:",names)
    print(f"Number of comparisons required for query {i}:",comparisions)

Query 1: investigation AND experimental AND wing
Number of documents retrieved for query 1: 19
Names of documents retrieved for query 1: cranfield0001.txt, cranfield0030.txt, cranfield0078.txt, cranfield0189.txt, cranfield0202.txt, cranfield0222.txt, cranfield0225.txt, cranfield0442.txt, cranfield0497.txt, cranfield0636.txt, cranfield0694.txt, cranfield0712.txt, cranfield1062.txt, cranfield1074.txt, cranfield1075.txt, cranfield1092.txt, cranfield1337.txt, cranfield1338.txt, cranfield1341.txt
Number of comparisions required for query 1: 679
